<a href="https://colab.research.google.com/github/safakan/CCTB_repo_safak_atakan_celik/blob/main/DEA113/SpamClassificationExercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



considerations for later:
- lemmatization instead of stemming?
- removing punctuation, there seems to be random punctuations

In [17]:
import pandas as pd
import numpy as np

In [18]:
import nltk

resources = ["punkt_tab", "stopwords"]
# vader_lexicon: SentimentIntensityAnalyzer()
for resource in resources:
    nltk.download(resource)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
data = [
    # Non-Spam (label = 1)
    {"text": "Excited to join the French newsletter to improve my language skills!", "label": 1},
    {"text": "Please enroll me! Can’t wait to start learning French grammar.", "label": 1},
    {"text": "As a cctb_student, need this newsletter for my coursework.", "label": 1},
    {"text": "Looking forward to weekly French vocabulary lessons!", "label": 1},
    {"text": "Your newsletter will help me prepare for my French exam. Thank you!", "label": 1},
    {"text": "cctb_student here – please add me to the mailing list.", "label": 1},
    {"text": "I’m eager to practice French writing with your tips.", "label": 1},
    {"text": "Interested in learning conversational French for my trip to Paris.", "label": 1},
    {"text": "This is exactly what I need to stay motivated while studying French!", "label": 1},
    {"text": "cctb_student registration: I’d love cultural tips alongside lessons.", "label": 1},
    {"text": "Thrilled to enhance my pronunciation with your expert guidance.", "label": 1},
    {"text": "Count me in! I want daily French practice emails.", "label": 1},
    {"text": "I’ve heard amazing things about your French learning resources.", "label": 1},
    {"text": "As a beginner, structured lessons via email would be perfect.", "label": 1},
    {"text": "cctb_student – I want to improve my reading comprehension!", "label": 1},
    {"text": "cctb_student – what's happening in this french newsletter my dear", "label": 1},
    # {"text": "I want to subscribe to the french newsletter to learn", "label": 1},
    # {"text": "I want to join because I want to learn", "label": 0},
    # {"text": "I want to be in, I plan to read and get exposed to french content", "label": 0},
    # {"text": "Let me in I just want to learn and continue", "label": 0},

    # Spam (label = 0)
    {"text": "Free iPhone giveaway! Click here to claim instantly.", "label": 0},
    {"text": "This service is useless. Don’t bother signing up.", "label": 0},
    {"text": "Buy cheap watches online – best deals guaranteed!", "label": 0},
    {"text": "I hate French; why would anyone learn this?", "label": 0},
    {"text": "Sign me up.", "label": 0},
    {"text": "Earn $500 daily with this secret method!", "label": 0},
    {"text": "Your newsletter sucks. Stop spamming me.", "label": 0},
    {"text": "Random text to test the system. Ignore.", "label": 0},
    {"text": "Promote your business with our marketing services!", "label": 0},
    {"text": "I didn’t ask for this. Remove me now.", "label": 0},
    {"text": "Who even uses French anymore? Useless.", "label": 0},
    {"text": "Get discounted software licenses here!", "label": 0},
    {"text": "This is a scam; nobody should subscribe.", "label": 0},
    {"text": "Why is the registration so complicated?!", "label": 0},
    {"text": "I’m just here to post comments, not learn.", "label": 0},
    {"text": "cctb_student, I don't even know what is this", "label": 0},
    {"text": "cctb_student, let me in because I have my reasons", "label": 0},
    {"text": "!@#$%^&*()_+{}|:\"<>?", "label": 0},
    {"text": "~~~~~~~~~~", "label": 0},
    {"text": "//////??????//////", "label": 0},
    {"text": "(*&^%$#@!)", "label": 0},
    {"text": ".........,,,,,,,,,,", "label": 0},
    {"text": "###########@@@@@@@", "label": 0},
    {"text": "<<<<>>>>====?????", "label": 0},
    {"text": "what's happening my dear", "label": 0},
    # {"text": "I want to join", "label": 0},
    # {"text": "I want to subscribe", "label": 0},
    # {"text": "I want to be in", "label": 0},
    # {"text": "Let me in", "label": 0}
]

df = pd.DataFrame(data)

In [20]:
# Tokenization
from nltk.tokenize import word_tokenize
df["tokens"] = df["text"].apply(lambda x: nltk.word_tokenize(x))


# # Word Counts
# from collections import Counter
# df["word_counts"] = df["tokens"].apply(lambda x: Counter(x))


# Filtering tokens: removing stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df["filtered_tokens"] = df["tokens"].apply(lambda x: [word for word in x if word.lower() not in stop_words])


# Stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df["stemmed_tokens"] = df["filtered_tokens"].apply(lambda x: [stemmer.stem(word) for word in x])


# Preprocessed text
df["preprocessed_text"] = df["stemmed_tokens"].apply(lambda x: " ".join(x))

In [21]:
# Text Classification Model Training 1
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB


# Training data
documents = list(df[["preprocessed_text", "label"]].itertuples(index=False, name=None))


# Prepare features and labels
count_vectorizer = CountVectorizer()
features = count_vectorizer.fit_transform([doc[0] for doc in documents])
labels = [doc[1] for doc in documents]


# Train a classifier (Naive Bayes)
NB_classifier = MultinomialNB()
NB_classifier.fit(features, labels)

MultinomialNB()

In [22]:
# Text Classification Model Training 2
from sklearn.feature_extraction.text import TfidfVectorizer
tfid_vectorizer = TfidfVectorizer(max_features=3000)


X = tfid_vectorizer.fit_transform(df["preprocessed_text"]).toarray()
y = df["label"]


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from sklearn.svm import SVC
SVC_classifier = SVC()
SVC_classifier.fit(X_train, y_train)

# from sklearn.metrics import accuracy_score,confusion_matrix,precision_score
# y_pred_svc = classifier.predict(X_test)
# accuracy_svc = accuracy_score(y_test, y_pred_svc)
# print(f"SVM Accuracy: {accuracy_svc:.2f}")
# print("confusion Matrix :",confusion_matrix(y_test,y_pred_svc))
# print("Precision Score: ",precision_score(y_test,y_pred_svc))

SVC()

In [44]:
#import randomforestclassifier

from sklearn.ensemble import RandomForestClassifier
RF_classifier = RandomForestClassifier()

RF_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)

# accuracy_rf = accuracy_score(y_test, y_pred_rf)
# print(f"Random Forest Accuracy: {accuracy_rf:.2f}")
# print("confusion Matrix :",confusion_matrix(y_test,y_pred_rf))
# print("Precision Score: ",precision_score(y_test,y_pred_rf))

In [45]:
rf_classifier

RandomForestClassifier()

---

## Spam Classification

In [41]:
# Test with an example
example_text = "I want in, I'm a learner, I want to practice and get exposed to content"

In [42]:
new_example = count_vectorizer.transform([example_text])
prediction = NB_classifier.predict(new_example)
print(prediction)

[1]


In [43]:
new_example = tfid_vectorizer.transform([example_text])
prediction = SVC_classifier.predict(new_example.toarray())
print(prediction)

[0]
